# Youtube_Content

In [1]:
import pandas as pd
import numpy as np

In [33]:
youtuber_df = pd.read_csv("youtuber_list.csv", encoding="utf-8")

In [34]:
# dataframe의 column명 변경
# youtuber_df.columns = ["name", "address"]

In [35]:
# youtuber_df["videos_ad"] = None ###초기화

In [36]:
youtuber_df.head(5)

,name,address,videos_ad
0,씬님,https://www.youtube.com/user/Hines382,https://www.youtube.com/user/Hines382/videos
1,윤짜미,https://www.youtube.com/user/yooncharmi,https://www.youtube.com/user/yooncharmi/videos
2,이사배,https://www.youtube.com/channel/UC9kmlDcqksaOn...,https://www.youtube.com/channel/UC9kmlDcqksaOn...
3,포니,https://www.youtube.com/channel/UCT-_4GqC-yLY1...,https://www.youtube.com/channel/UCT-_4GqC-yLY1...
4,Lena's Pocket Beauty,https://www.youtube.com/channel/UCU8mou_JjcF-I...,https://www.youtube.com/channel/UCU8mou_JjcF-I...


In [6]:
# youtuber_df

In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [8]:
cnt_youtuber = len(youtuber_df)

In [23]:
#if you want reset videos_info list
#reset_info = []
#videos_info = reset_info

In [24]:
#if you want reset videos_info list
#videos_info = []
#youtuber_videos = pd.DataFrame(videos_info, columns = ["youtuber_name", "video_id"])

In [41]:
youtuber_videos_df = pd.read_csv("youtuber_videos.csv", encoding = "utf-8")

In [42]:
youtuber_videos_df.head(5)

,youtuber_name,video_id
0,씬님,/watch?v=UTXuSv8e18E
1,씬님,/watch?v=_BsClDZgRaY
2,씬님,/watch?v=u_aOZxEiIFs
3,씬님,/watch?v=4ZVexkbtRLk
4,씬님,/watch?v=mXaHpDcESng


In [12]:
youtube = "https://www.youtube.com"

In [13]:
import time
from slacker import Slacker

In [14]:
slack = Slacker(token)

In [15]:
def get_address_bs(address): #입력값의 bs를 준다.
    driver = webdriver.Firefox()
    open_address = driver.get(address)
    time.sleep(20)
    get_source = driver.page_source
    address_bs = BeautifulSoup(get_source, "html.parser")
    driver.quit()
    return address_bs

In [16]:
def find_youtuber_videos_address(address_bs):
    while(1):
        try:
            youtuber_videos = address_bs.find_all("ul", {"id" : "channel-navigation-menu"})
            youtuber_videos = str(youtuber_videos)
            youtuber_videos = youtuber_videos.split("\n")
            youtuber_videos = youtuber_videos[5]
            youtuber_videos = youtuber_videos.split("href=\"")
            youtuber_videos = youtuber_videos[1]
            youtuber_videos = youtuber_videos[0:-55]
            youtuber_videos_address = youtube + youtuber_videos
            return youtuber_videos_address
        except IndexError:
            pass        

In [17]:
def videos_list_read_more_and_bs(videos_ad):
    #오류가 났을 시 shutdown되기 때문에 따로 빼기
    driver = webdriver.Firefox()
    open_videos_ad = driver.get(videos_ad)
    time.sleep(30)
    while(1):
        try:
            result = driver.find_element_by_class_name("load-more-text").click()
            time.sleep(2)
        except Exception:
            #pass pass를 할 경우 아예 무시가 된다.
            break
    videos_ad_source = driver.page_source
    videos_ad_bs = BeautifulSoup(videos_ad_source, "html.parser")
    driver.quit()
    return videos_ad_bs

In [18]:
def get_youtuber_videos_list(youtuber_name, videos_bs):
    while(1):
        videos_element = videos_bs.find_all("a", {"class" : "yt-uix-sessionlink yt-uix-tile-link spf-link yt-ui-ellipsis yt-ui-ellipsis-2"})
        videos_element = str(videos_element)
        videos_element = videos_element.split("</a>")
        cnt_video = len(videos_element)-1
        print(youtuber_name, "Total: ", cnt_video)
        print("\n\n\n")
        if not cnt_video == 0:
            break #비디오 개수가 0이 나오는 에러 해결
    for i in range(cnt_video):
        #video_name = videos_element[i]
        #video_name = video_name.split("title=\"")
        #video_name = video_name[1]
        #video_name = video_name.split("\">")
        #video_name = video_name[0]
        video_id = videos_element[i]
        video_id = video_id.split(" href=\"")
        video_id = video_id[1]
        video_id = video_id.split("\" title=")
        video_id = video_id[0]
        #video_info = [youtuber_name, video_id, video_name]
        video_info = [youtuber_name, video_id]
        if video_info not in videos_info:
            videos_info.append(video_info)
    message = "[" + youtuber_name + ", " + str(cnt_video) + "]"
    #slack.chat.post_message('#crawling', message)    
    time.sleep(3)

In [19]:
def main():
    for i in range(cnt_youtuber):
        name = youtuber_df["name"][i]
        address = youtuber_df["address"][i]
        if youtuber_df["videos_ad"][i] is None:
            ad_bs = get_address_bs(address)
            videos_ad = find_youtuber_videos_address(ad_bs)
            youtuber_df["videos_ad"][i] = videos_ad
            youtuber_df.to_csv("youtuber_list.csv", encoding="utf-8", index=False)
            videos_bs = videos_list_read_more_and_bs(videos_ad)
        else:
            videos_bs = videos_list_read_more_and_bs(youtuber_df["videos_ad"][i])
        #print(videos_bs)
        get_youtuber_videos_list(name, videos_bs)

In [20]:
main()

씬님 Total:  405




윤짜미 Total:  180




이사배 Total:  484




포니 Total:  47




Lena's Pocket Beauty Total:  115




영주 Total:  188




윤새 Total:  149




회사원(calarygirl) Total:  182




라뮤끄 Total:  185




곽토리 Total:  225




김기수 Total:  111




dayeong's beauty drawing Total:  214




루나(루나의 알파벳) Total:  59




꽁지 Total:  206




Yojin 조효진 Total:  56




YENNY 예니 Total:  120




썸블리 Total:  148




EVA 에바 Total:  112




써니채널 Sunny's Channel Total:  88




Daddoa 다또아 Total:  131




yeondukong 연두콩 Total:  180




Yoo True Total:  495






In [22]:
len(videos_info)

4080

In [38]:
youtuber_videos_df = pd.DataFrame(videos_info, columns = ["youtuber_name", "video_id"])

In [39]:
youtuber_videos_df.to_csv("youtuber_videos.csv", index=False, encoding="utf-8")

In [162]:
#df2 = df1.drop_duplicates("컨텐츠명") column 삭제